# Chapter2 提供推荐

下面从源码中导入相关的评论的数据.

In [6]:
import recommendations
from pprint import pprint
pprint(recommendations.critics)

{'Claudia Puig': {'Just My Luck': 3.0,
                  'Snakes on a Plane': 3.5,
                  'Superman Returns': 4.0,
                  'The Night Listener': 4.5,
                  'You, Me and Dupree': 2.5},
 'Gene Seymour': {'Just My Luck': 1.5,
                  'Lady in the Water': 3.0,
                  'Snakes on a Plane': 3.5,
                  'Superman Returns': 5.0,
                  'The Night Listener': 3.0,
                  'You, Me and Dupree': 3.5},
 'Jack Matthews': {'Lady in the Water': 3.0,
                   'Snakes on a Plane': 4.0,
                   'Superman Returns': 5.0,
                   'The Night Listener': 3.0,
                   'You, Me and Dupree': 3.5},
 'Lisa Rose': {'Just My Luck': 3.0,
               'Lady in the Water': 2.5,
               'Snakes on a Plane': 3.5,
               'Superman Returns': 3.5,
               'The Night Listener': 3.0,
               'You, Me and Dupree': 2.5},
 'Michael Phillips': {'Lady in the Water': 2.5,
    

## 2.1寻找相近用户
这里我们主要利用**欧几里德距离**和**皮尔逊相关度**来计算用户之间的相关性，根据相关性的高低来寻找相近用户
### 2.1.1 欧几里德距离
这里的[欧几里德距离](https://zh.wikipedia.org/zh-hans/%E6%AC%A7%E5%87%A0%E9%87%8C%E5%BE%97%E8%B7%9D%E7%A6%BB)就是我们上学的时候在直角坐标系里面求两点距离。

In [7]:
from math import sqrt
sqrt(pow(4.5-4, 2) + pow(1-2,2))
1/(1+sqrt(pow(4.5-4, 2) + pow(1-2,2)))

def sim_distance(prefs, person1, person2):
    share_items = set(prefs[person1])&set(prefs[person2])
    if not share_items:
        return 0
    else:
        sum_of_squares = sum([pow(prefs[person1][item]-prefs[person2][item],2) for item in share_items])
        return 1/(1+sqrt(sum_of_squares))

In [8]:
sim_distance(recommendations.critics, "Lisa Rose", "Gene Seymour")

0.29429805508554946

### 2.1.2 皮尔逊相关性
皮尔逊相关性系数用于评价两组数据与某一直线拟合程序。其计算过程相比上面的欧几里德距离复杂，在数据与平均水平偏差比较大的时候，得到的结果会比欧几里德距离更好。因为皮尔逊相关性在计算时，会修正“夸大分值”的情况，而在欧几里德距离计算中，夸大分值会拉大距离结果。网上关于皮尔逊相关性系数的[链接1](https://www.zhihu.com/question/19734616)，[链接2](https://segmentfault.com/q/1010000000094674)。

In [9]:
def sim_pearson(prefs, p1, p2):
    share_items = set(prefs[p1])&set(prefs[p2])
    if len(share_items) == 0:
        return 1
    
    n = len(share_items)
    sum1 = sum([prefs[p1][it] for it in share_items])
    sum2 = sum([prefs[p2][it] for it in share_items])
    
    sum1Sq = sum([pow(prefs[p1][it],2) for it in share_items])
    sum2Sq = sum([pow(prefs[p2][it],2) for it in share_items])
    
    pSum = sum([prefs[p1][it]*prefs[p2][it] for it in share_items])
    
    num = pSum - (sum1*sum2/n)
    den = sqrt( (sum1Sq-pow(sum1,2)/n) * (sum2Sq-pow(sum2,2)/n) )
    if den == 0:
        return 0
    return num/den

In [10]:
sim_pearson(recommendations.critics, "Lisa Rose", "Gene Seymour")

0.39605901719066977

其他相似度的计算方法有**Jaccard系数**，**曼哈顿系数**等。几种常用相似性方法的Python实现[链接](http://dataaspirant.com/2015/04/11/five-most-popular-similarity-measures-implementation-in-python/).

## 2.2 为评论者打分
根据上面写的相似度评估方法，我们来寻找指定人员与其他人之间的匹配程序，找出最佳匹配人。

In [43]:
def topMatches(prefs, person, n=5, similarity=sim_pearson):
    scores = [(similarity(prefs, person, other), other) for other in prefs if other!=person]
    scores.sort()
    scores.reverse()
    return scores[0:n]

In [27]:
# 找出与Toby品位匹配最高的3个人
sim_pearson(recommendations.critics, 'Toby', 'Lisa Rose')
print topMatches(recommendations.critics, 'Toby')
print topMatches(recommendations.critics, 'Toby', similarity=sim_pearson)

[(0.4, 'Mick LaSalle'), (0.38742588672279304, 'Michael Phillips'), (0.3567891723253309, 'Claudia Puig'), (0.3483314773547883, 'Lisa Rose'), (0.2674788903885893, 'Jack Matthews')]
[(0.9912407071619299, 'Lisa Rose'), (0.9244734516419049, 'Mick LaSalle'), (0.8934051474415647, 'Claudia Puig'), (0.66284898035987, 'Jack Matthews'), (0.38124642583151164, 'Gene Seymour')]


可以看到不同的相似性分析方法，得到的结果也是不一样的，而且感觉差异还是挺大的。
## 2.3 推荐物品
上面的例子，我们可以获得与自身品位相似的用户，可以通过查找他看过的电影来寻找我们感兴趣的电影。可是如果有一部电影没有被他们看过，而我们又刚好感兴趣时该怎么办呢。这时就需要一个方法把所有感兴趣的电影列出来。采用的计算方法是以每个人的相似度为权重系数，乘以该人对电影的评分得到一个经过矫正后的评分，再把这些人的矫正评分求和。为了防止因为某部电影因为评论人数多而对结果造成影响，还需要把矫正评分的求和结果除以每人相似度的求和。

In [32]:
def getRecommendations(prefs, person, similarity=sim_pearson):
    totals = {}
    simSums = {}
    for other in prefs:
        if other == person:
            continue
        sim = similarity(prefs, person, other)
        
        if sim<=0:
            continue
        for item in prefs[other]:
            if item not in prefs[person] or prefs[person][item] == 0:
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
                simSums.setdefault(item,0)
                simSums[item]+=sim
    rankings=[(total/simSums[item], item) for item, total in totals.items()]
    rankings.sort()
    rankings.reverse()
    return rankings

In [33]:
getRecommendations(recommendations.critics, 'Toby')

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

上面得到的列表，我们可以理解成Toby对没看过的电影可能打出的评分。打分高的电影，当然更适合Toby的口味。

## 2.4 匹配物品  
前面我们已经知道如何查找品位相似的人，下面是想找相似商品。大体的思路是把前面推荐人的算法里面，把人和物品的位置交换一下。首先需要把`recommendations.critics`里面保存的信息转换一下。

In [36]:
def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            result[item][person] = prefs[person][item]
    return result       

In [44]:
movies = transformPrefs(recommendations.critics)
#print movies
print topMatches(movies, 'Superman Returns', similarity=sim_pearson)
print getRecommendations(movies, 'Just My Luck')


[(0.6579516949597695, 'You, Me and Dupree'), (0.4879500364742689, 'Lady in the Water'), (0.11180339887498941, 'Snakes on a Plane'), (-0.1798471947990544, 'The Night Listener'), (-0.42289003161103106, 'Just My Luck')]
[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]


书上说，把人和物品对调并不是总能得到有意义的结果，但多大数情况下有助于我们做出有意义的对比。

## 2.5 构建基于del.icio.us的链接推荐系统
[Delicious](https://del.icio.us/)是一个链接推荐网站，你需要把你感兴趣的链接张贴上去，然后会给你推荐你可能感兴趣的链接。主要思路类似上面的物品推荐功能。  
这里我们直接利用现成的Python API来获取数据，在下面的代码，直接导入。

In [45]:
import pydelicious
pydelicious.get_popular(tag='programming')

ImportError: No module named feedparser